In [1]:
import torch

### Tensor Manipulation

In [16]:
lst = [[1,2,3],[4,5,6]]
tens = torch.tensor(lst, dtype=torch.float)
tens

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [6]:
shape = (3,3)
ones = torch.ones(shape)
zeros = torch.zeros(shape)
random_tensor = torch.rand(shape)

ones,zeros,random_tensor

(tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]),
 tensor([[0.9299, 0.6951, 0.0482],
         [0.5849, 0.8463, 0.5416],
         [0.7120, 0.5172, 0.9348]]))

In [7]:
random_like = torch.rand_like(tens, dtype = torch.float)
random_like

tensor([[0.3669, 0.4170, 0.3361],
        [0.4436, 0.0118, 0.3333]])

In [8]:
random_like.shape, random_like.requires_grad

(torch.Size([2, 3]), False)

### Autograd

In [13]:
a = torch.tensor(2.0, requires_grad = True)
b = torch.tensor(3.0, requires_grad = True)
x = torch.tensor(4.0, requires_grad = True)
y= a+b
z = y * x
y, z

(tensor(5., grad_fn=<AddBackward0>), tensor(20., grad_fn=<MulBackward0>))

In [14]:
y. grad_fn, z.grad_fn, a.grad_fn

(<AddBackward0 at 0x268b0a5b7c0>, <MulBackward0 at 0x268b0a598a0>, None)

### Reduction

In [18]:
row_mean = tens.mean(dim = 1) #Collapse columns
col_mean = tens.mean(dim = 0) #Collapse rows
row_mean, col_mean

(tensor([2., 5.]), tensor([2.5000, 3.5000, 4.5000]))

### Advanced Indexing and Gather

In [25]:
indices = torch.tensor([
    [1,0], # row 1
    [0,2],
    [2,1] #row 3
])

vals_rowise = torch.gather(random_tensor, dim = 1, index = indices) #along columns -> rowise
vals_colwise = torch.gather(random_tensor, dim = 0, index = indices) #along rows -> colwise 
indices, vals_rowise, vals_colwise

(tensor([[1, 0],
         [0, 2],
         [2, 1]]),
 tensor([[0.6951, 0.9299],
         [0.5849, 0.5416],
         [0.9348, 0.5172]]),
 tensor([[0.5849, 0.6951],
         [0.9299, 0.5172],
         [0.7120, 0.8463]]))

### Simple Model

In [36]:
### Create True Data

N = 10 #rows
D_in = 2
D_out = 1

X = torch.randn(N, D_in)

true_W = torch.tensor([[2.0], [3.0]])
true_b = torch.tensor(1.0)

y_true = X @ true_W + true_b + (0.1 * torch.rand(N,D_out)) #little noise

true_W.shape, y_true.shape, y_true

(torch.Size([2, 1]),
 torch.Size([10, 1]),
 tensor([[-3.4866],
         [-2.1583],
         [ 5.5293],
         [-4.5631],
         [-1.0018],
         [ 1.3235],
         [ 4.1933],
         [ 5.1034],
         [-5.5167],
         [ 1.0321]]))

### Single Epoch

In [65]:
### Init Params
W = torch.randn(D_in, D_out, requires_grad = True)
b = torch.randn(1, requires_grad = True)

W, b

(tensor([[0.2695],
         [0.3662]], requires_grad=True),
 tensor([0.1883], requires_grad=True))

In [66]:
y_hat = X @ W + b
y_hat

tensor([[-0.3979],
        [-0.1873],
        [ 0.7821],
        [-0.5058],
        [-0.0919],
        [ 0.2306],
        [ 0.6089],
        [ 0.7032],
        [-0.6042],
        [ 0.1817]], grad_fn=<AddBackward0>)

In [67]:
error = y_hat - y_true
squared_error = error ** 2
loss = squared_error.mean()
loss = torch.mean((y_hat-y_true)**2)
loss

tensor(11.1512, grad_fn=<MeanBackward0>)

In [68]:
loss.backward()
W.grad, b.grad

(tensor([[-4.5578],
         [-5.4638]]),
 tensor([0.0528]))

### Multiple Epochs

In [74]:
EPOCHS = 300; lr = 0.01

### Init Params
W = torch.randn(D_in, D_out, requires_grad = True)
b = torch.randn(1, requires_grad = True)

for epoch in range(EPOCHS):
    #forward pass
    y_hat = X @ W + b

    #loss
    loss = torch.mean((y_hat - y_true)**2)

    #backprop
    loss.backward()

    #Update Params
    with torch.no_grad():
        W -= (lr * W.grad)
        b -= (lr*b.grad)

    if ((epoch+1)%30 == 0):
        print(f"Epoch {epoch+1}: Loss:{loss.item():.4f} | W = {W} | b = {b.item():.3f}")

    #zero gradients to prevent accumulation
    W.grad.zero_(); b.grad.zero_()   
    

Epoch 30: Loss:2.2511 | W = tensor([[1.0814],
        [2.1025]], requires_grad=True) | b = 0.369
Epoch 60: Loss:0.6285 | W = tensor([[1.6119],
        [2.4526]], requires_grad=True) | b = 0.592
Epoch 90: Loss:0.1957 | W = tensor([[1.8605],
        [2.6535]], requires_grad=True) | b = 0.730
Epoch 120: Loss:0.0716 | W = tensor([[1.9725],
        [2.7726]], requires_grad=True) | b = 0.821
Epoch 150: Loss:0.0313 | W = tensor([[2.0191],
        [2.8461]], requires_grad=True) | b = 0.884
Epoch 180: Loss:0.0159 | W = tensor([[2.0354],
        [2.8933]], requires_grad=True) | b = 0.928
Epoch 210: Loss:0.0089 | W = tensor([[2.0382],
        [2.9248]], requires_grad=True) | b = 0.960
Epoch 240: Loss:0.0053 | W = tensor([[2.0356],
        [2.9466]], requires_grad=True) | b = 0.984
Epoch 270: Loss:0.0034 | W = tensor([[2.0312],
        [2.9621]], requires_grad=True) | b = 1.001
Epoch 300: Loss:0.0023 | W = tensor([[2.0266],
        [2.9733]], requires_grad=True) | b = 1.015


In [75]:
print(f"Params Found: W = {W} | b = {b.item():.3f}")
print(f"True Params: W_true = {true_W} | b_true = {true_b.item():.3f}")

Params Found: W = tensor([[2.0266],
        [2.9733]], requires_grad=True) | b = 1.015
True Params: W_true = tensor([[2.],
        [3.]]) | b_true = 1.000
